In [3]:
pip install azure-storage-file-datalake pandas pyarrow openpyxl xlrd  matplotlib  python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [3]:
from azure.storage.filedatalake import DataLakeServiceClient
import pandas as pd
import os
from io import BytesIO
from dotenv import load_dotenv
from pathlib import Path

load_dotenv()


# Access the environment variables
account_name = os.getenv('ACCOUNT_NAME')
container = os.getenv('CONTAINER')
sas_token = os.getenv('SAS_TOKEN')
relative_path = os.getenv('RELATIVE_PATH')

service_client = DataLakeServiceClient(account_url=f"https://{account_name}.dfs.core.windows.net", credential=sas_token)

file_system_client = service_client.get_file_system_client(container)


dataframes = []
paths = file_system_client.get_paths(path=relative_path) 
for path in paths:
    file_path = path.name  # Ensure you are using the correct file path from the directory listing
    print(f"Attempting to download: {file_path}")
    
    try:
        file_client = file_system_client.get_file_client(file_path)
        download = file_client.download_file()
        downloaded_bytes = download.readall()
        print(f"Successfully downloaded: {file_path}")
        if file_path.endswith(".xlsx"):
            df = pd.read_excel(BytesIO(downloaded_bytes), engine='openpyxl',skiprows=3)
        elif file_path.endswith(".xls"):
            df = pd.read_excel(BytesIO(downloaded_bytes), engine='xlrd',skiprows=3)
        
        # Convert the 'Session' column to numeric, forcing errors to NaN
        df['Session'] = pd.to_numeric(df['Session'], errors='coerce')
            
        # Filter out rows where 'Session' is NaN (non-numeric)
        df = df.dropna(subset=['Session'])

        dataframes.append(df)
        print(f"Successfully downloaded and read: {file_path}")
    except Exception as e:
        print(f"Error downloading {file_path}: {str(e)}")

if dataframes:
    combined_df = pd.concat(dataframes, ignore_index=True)
    for col in combined_df.columns:
        combined_df[col] = combined_df[col].astype(str)
    combined_df.rename(columns=lambda x: x.strip().replace(' ', '_').replace('.', '').replace('/', '_'), inplace=True)


    # Save the combined DataFrame as a Parquet file
    parquet_file_path = "combined_data.parquet"
    combined_df.to_parquet(parquet_file_path, engine='pyarrow')
    print(f"Successfully saved combined DataFrame to '{parquet_file_path}'")
else:
    print("No Excel files found to process.")




Attempting to download: deltalake/PassportIQ/bronze/CASTransactionSummary Aug 01 2023- Aug 20 2023.xls
Successfully downloaded: deltalake/PassportIQ/bronze/CASTransactionSummary Aug 01 2023- Aug 20 2023.xls
Successfully downloaded and read: deltalake/PassportIQ/bronze/CASTransactionSummary Aug 01 2023- Aug 20 2023.xls
Attempting to download: deltalake/PassportIQ/bronze/CASTransactionSummary Aug 20 2023- Aug 31 2023.xls
Successfully downloaded: deltalake/PassportIQ/bronze/CASTransactionSummary Aug 20 2023- Aug 31 2023.xls
Successfully downloaded and read: deltalake/PassportIQ/bronze/CASTransactionSummary Aug 20 2023- Aug 31 2023.xls
Attempting to download: deltalake/PassportIQ/bronze/CASTransactionSummary July 01 2023-July 20 2023.xls
Successfully downloaded: deltalake/PassportIQ/bronze/CASTransactionSummary July 01 2023-July 20 2023.xls
Successfully downloaded and read: deltalake/PassportIQ/bronze/CASTransactionSummary July 01 2023-July 20 2023.xls
Attempting to download: deltalake/Pas